# Implementing a neural network in Python

---

## Programming Environment

```bash
conda create -n tf -v -y \
-c conda-forge --strict-channel-priority \
'imbalanced-learn=0.11' \
'matplotlib      =3.07' \
'numpy           =1.25' \
'pandas          =2.01' \
'plotly          =5.16' \
'python          =3.11' \
'scikit-learn    =1.03'
```
```bash
conda activate tf
```
[Get started with tensorflow-metal](https://developer.apple.com/metal/tensorflow-plugin/)
```bash
python -m pip install tensorflow
```
```bash
python -m pip install tensorflow-metal
```

In [1]:
import numpy  as np
np.set_printoptions(suppress=True) # suppress scientific notation
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from   sklearn.model_selection import train_test_split
from   sklearn.preprocessing   import StandardScaler
import imblearn

import tensorflow as tf

import plotly

import datetime
from   importlib.metadata import version
import os
import platform as p
import sys

pad = 20
print(  f"\n{'Executed' : <{pad}} : {datetime.datetime.now().astimezone().strftime('%Y-%m-%d %H:%M:%S %z %Z')}"
        f"\n{'Platform' : <{pad}} : {p.platform(aliased = False, terse = False)}"
        f"\n{'Conda'    : <{pad}} : {os.environ['CONDA_DEFAULT_ENV'] or sys.executable.split('/')[-3]}"
        f"\n{'Python'   : <{pad}} : {p.python_implementation()} {p.python_version()} {sys.executable}")
print(*[f'{name : <{pad}} : {version(name)}'
        for name in ['Imbalanced-Learn', 'Matplotlib', 'NumPy', 'Pandas', 'Plotly', 'Scikit-Learn', 'TensorFlow']], sep = '\n')


Executed             : 2023-09-03 15:12:57 -0400 EDT
Platform             : macOS-13.5.1-arm64-arm-64bit
Conda                : ml
Python               : CPython 3.11.4 /Users/df/anaconda3/envs/ml/bin/python
Imbalanced-Learn     : 0.11.0
Matplotlib           : 3.7.2
NumPy                : 1.24.3
Pandas               : 2.0.3
Plotly               : 5.15.0
Scikit-Learn         : 1.3.0
TensorFlow           : 2.13.0


---

In [2]:
application_df = pd.read_csv(filepath_or_buffer = 'https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df = application_df.drop(labels = ['EIN', 'NAME'],
                                     axis   = 1)
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
application_types_to_replace = list(application_counts[application_counts < 500].index)
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [6]:
for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(to_replace = app, value = 'Other')

application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
class_counts[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
class_counts[class_counts > 10]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: count, dtype: int64

In [10]:
classifications_to_replace = list(class_counts[class_counts < 1000].index)
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [11]:
for cls in classifications_to_replace:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(to_replace = cls, value = 'Other')

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [12]:
application_with_dummies_df = pd.get_dummies(data = application_df)
application_with_dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


In [13]:
X = application_with_dummies_df.drop(labels = ['IS_SUCCESSFUL'], axis = 'columns').values
y = application_with_dummies_df['IS_SUCCESSFUL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78)

print(X      .shape)
print(y      .shape)
print(X_train.shape)
print(X_test .shape)
print(y_train.shape)
print(y_test .shape)

(34299, 43)
(34299,)
(25724, 43)
(8575, 43)
(25724,)
(8575,)


In [14]:
scaler = StandardScaler()
scaler

StandardScaler()

In [15]:
X_scaler = scaler.fit(X = X_train)
X_scaler

StandardScaler()

In [16]:
X_train_scaled = X_scaler.transform(X = X_train)
X_train_scaled.shape

(25724, 43)

In [17]:
X_test_scaled = X_scaler.transform(X = X_test)
X_test_scaled.shape

(8575, 43)

In [18]:
number_input_features = len(X_train[0]) # 43
hidden_nodes_layer1   = 80
hidden_nodes_layer2   = 30

nn = tf.keras.models.Sequential()
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer1,
                                     input_dim  = number_input_features,
                                     activation = 'relu'))
nn.add(layer = tf.keras.layers.Dense(units      = hidden_nodes_layer2,
                                     activation = 'relu'))
nn.add(layer = tf.keras.layers.Dense(units      = 1,
                                     activation = 'sigmoid'))
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-09-03 15:12:58.643020: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-09-03 15:12:58.643043: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-09-03 15:12:58.643048: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-09-03 15:12:58.643087: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-03 15:12:58.643104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [22]:
nn.compile(optimizer = 'adam',
           loss      = 'binary_crossentropy',
           metrics   = ['accuracy'])

fit_model = nn.fit(x = X_train_scaled,
                   y = y_train,
                   epochs = 100)

Epoch 1/100


2023-09-03 15:14:46.254118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 8s 8ms/step - loss: 0.6019 - accuracy: 0.7093
Epoch 2/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6009 - accuracy: 0.7100
Epoch 3/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6246 - accuracy: 0.6990
Epoch 4/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6499 - accuracy: 0.6843
Epoch 5/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6675 - accuracy: 0.6800
Epoch 6/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6904 - accuracy: 0.6742
Epoch 7/100
804/804 [==============================] - 6s 7ms/step - loss: 0.7001 - accuracy: 0.6705
Epoch 8/100
804/804 [==============================] - 6s 7ms/step - loss: 0.7874 - accuracy: 0.6536
Epoch 9/100
804/804 [==============================] - 6s 8ms/step - loss: 0.7789 - accuracy: 0.6531
Epoch 10/100
804/804 [==============================] - 6s 8ms/step - loss: 0.8167 - accuracy: 0.6502
E

In [23]:
model_loss, model_accuracy = nn.evaluate(x = X_test_scaled,
                                         y = y_test,
                                         verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2023-09-03 15:26:31.868922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 10.8782 - accuracy: 0.7180 - 1s/epoch - 5ms/step
Loss: 10.878178596496582, Accuracy: 0.7180175185203552


In [26]:
nn.save(filepath = 'AlphabetSoupCharity.keras')
#nn.save('AlphabetSoupCharity.h5')

---